In [1]:
import numpy as np

In [18]:
from src.processing.features.zyskowski import ZyskowskiFeatureExtractor
from src.processing.process import process
from src.model.data.dataset import DuelDataset

data = process(path="../../../res/json", extractor=ZyskowskiFeatureExtractor())
dataset = DuelDataset(features=data[0], labels=data[1])

error parsing round: 'NoneType' object is not iterable
error parsing round: 'NoneType' object is not iterable


In [20]:
print(dataset.x_temporal.shape)
print(dataset.x_snapshot.shape)
print(dataset.y.shape)

torch.Size([2690, 14, 4])
torch.Size([2690, 80])
torch.Size([2690])


In [21]:
kernel = np.array([0.1, 0.2, 0.3, 0.5])


def get_xy(dataset: DuelDataset):
    x_temporal = np.sum(dataset.x_temporal.numpy() * kernel, axis=2)
    x_snapshot = dataset.x_snapshot.numpy()
    x = np.concatenate((x_temporal, x_snapshot), axis=1)
    y = dataset.y.numpy()
    return x, y


x, y = get_xy(dataset)
print(x.shape, y.shape)

(2690, 94) (2690,)


In [23]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(2152, 94) (2152,) (538, 94) (538,)


In [24]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=0)
logreg.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [25]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

y_pred = logreg.predict(x_train)
print(accuracy_score(y_train, y_pred))
print(f1_score(y_train, y_pred))

tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)

0.7644052044609665
0.7751662971175166
771 258 249 874


In [26]:
print(logreg.coef_)

[[-0.0750852   0.13895651 -0.18206282  0.10599457 -0.01594868 -0.00815956
  -0.02430634  0.03077568  0.08959198  0.02067421  0.11350087 -0.09710423
  -1.06543064  0.61802464 -0.36300736 -0.00627113 -0.29187154  0.31601075
   0.18427154  0.07587247 -0.00271724  0.25798449 -0.01883521  0.31089186
   0.2636374   0.13752821  0.09346293 -0.06780596 -0.06111248  0.03038285
   0.33613726  0.05564141  0.00781499  0.          0.19754372  0.0422553
   0.32905948  0.07860817 -0.00807467  0.26121994  0.06025279  0.15248108
   0.08406795 -0.11536112 -0.07808124  0.11727899  0.13729906 -0.03842893
   0.00673351 -0.28982543  0.01478318  0.09994672 -0.23790625 -0.27304318
  -0.20863396  0.05156778  0.18423105 -0.14232671 -0.11616102  0.
  -0.11638289  0.09636429  0.02633463  0.00355263  0.01079762  0.1230617
  -0.35281927 -0.11099816  0.08334047 -0.23883403  0.18973739  0.21797489
  -0.3938182   0.37113139 -0.12998684  0.27535126  0.12381139 -0.10900984
   0.2214695  -0.31306872  0.00928149 -0.0054464

In [27]:
y_pred = logreg.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)

0.7657992565055762
0.7709090909090909
200 55 71 212
